# Veri Ölçeklendirme: Normalizasyon ve Standardizasyon Rehberi

Bu rehber, makine öğrenmesi ön işleme adımlarının en önemlilerinden olan **veri ölçeklendirme (feature scaling)** konusunu ele alır. `Scikit-learn` kütüphanesi kullanarak en yaygın iki yöntem olan **Normalizasyon (Min-Max Scaling)** ve **Standardizasyon (Z-score Scaling)** arasındaki farkları, nasıl uygulandıklarını ve ne zaman hangisini tercih etmeniz gerektiğini öğreneceksiniz.

## 1. Veri Ölçeklendirme Neden Gereklidir?

Makine öğrenmesi algoritmalarının birçoğu, özelliklerin (features) farklı ölçeklerde olmasından olumsuz etkilenir. Örneğin, bir veri setinde **yaş** (18-65 arası) ve **maaş** (30.000-250.000 arası) gibi iki özellik olduğunu düşünelim.

- **Mesafe Bazlı Algoritmalar (KNN, K-Means, SVM):** Bu algoritmalar, noktalar arasındaki mesafeyi hesaplarken, daha büyük ölçekli olan `maaş` özelliği, `yaş` özelliğini domine eder ve modelin yanlı sonuçlar üretmesine neden olur.
- **Gradyan İnişi (Gradient Descent) Kullanan Algoritmalar (Lineer/Lojistik Regresyon, Sinir Ağları):** Ölçeklendirme, gradyan inişinin daha hızlı ve kararlı bir şekilde optimum noktaya ulaşmasına yardımcı olur.

**Amaç:** Tüm sayısal özellikleri benzer bir ölçeğe getirerek modelin tüm özellikleri adil bir şekilde değerlendirmesini sağlamaktır.

## 2. Kurulum ve Örnek Veri Seti

Farklı ölçek ve dağılımlara sahip sentetik bir veri seti oluşturalım ve bu veri üzerinde ölçeklendirmenin etkilerini görselleştirelim.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Sentetik veri oluşturalım
np.random.seed(42)
data = {
    'yas': np.random.randint(20, 61, 100),
    'deneyim_yili': np.random.randint(0, 40, 100),
    'maas': np.random.randint(30000, 150001, 100),
    # Aykırı değerler içeren bir özellik
    'bonus': np.concatenate([np.random.randint(1000, 5000, 95), np.array([15000, 20000, 22000, 25000, 30000])])
}
df = pd.DataFrame(data)

# Orijinal verinin dağılımlarını görselleştirelim
sns.kdeplot(data=df, palette='viridis')
plt.title('Ölçeklendirme Öncesi Orijinal Dağılımlar')
plt.show()

display(df.describe().round(2))

## 3. Yöntem 1: Normalizasyon (Min-Max Scaling)

Bu yöntem, veriyi belirli bir aralığa, genellikle **[0, 1]** aralığına, sıkıştırır.

**Formül:** `X_normalized = (X - X_min) / (X_max - X_min)`

- **Avantajları:** Verinin belirli sınırlar içinde olmasını garanti eder. Görüntü işleme veya sinir ağlarında belirli aktivasyon fonksiyonları için kullanışlıdır.
- **Dezavantajları:** Aykırı değerlere (outliers) karşı çok hassastır. Tek bir aykırı değer, verinin geri kalanının çok dar bir aralığa sıkışmasına neden olabilir.

In [ ]:
scaler_minmax = MinMaxScaler()
df_minmax_scaled = pd.DataFrame(scaler_minmax.fit_transform(df), columns=df.columns)

# Min-Max ölçeklendirme sonrası dağılımlar
sns.kdeplot(data=df_minmax_scaled, palette='viridis')
plt.title('Min-Max Normalizasyon Sonrası Dağılımlar')
plt.show()

print("Min-Max Normalizasyon Sonrası İstatistikler:")
display(df_minmax_scaled.describe().round(2))

## 4. Yöntem 2: Standardizasyon (Z-score Scaling)

Bu yöntem, veriyi ortalaması **0** ve standart sapması **1** olan bir dağılıma dönüştürür.

**Formül:** `X_standardized = (X - μ) / σ` (μ: ortalama, σ: standart sapma)

- **Avantajları:** Aykırı değerlerden Normalizasyon kadar etkilenmez. Verinin orijinal dağılımının şeklini korur. Çoğu makine öğrenmesi algoritması için **genellikle varsayılan ve en güvenli tercihtir**.
- **Dezavantajları:** Veriyi belirli bir aralığa sıkıştırmaz.

In [ ]:
scaler_standard = StandardScaler()
df_standard_scaled = pd.DataFrame(scaler_standard.fit_transform(df), columns=df.columns)

# Standardizasyon sonrası dağılımlar
sns.kdeplot(data=df_standard_scaled, palette='viridis')
plt.title('Standardizasyon Sonrası Dağılımlar')
plt.show()

print("Standardizasyon Sonrası İstatistikler:")
display(df_standard_scaled.describe().round(2))

## 5. Yöntem 3: Dayanıklı Ölçeklendirme (Robust Scaling)

Veri setinizde **önemli aykırı değerler** varsa, `RobustScaler` en iyi seçenek olabilir. Ortalama ve standart sapma yerine, **medyan** ve **çeyrekler arası aralığı (IQR)** kullandığı için aykırı değerlerden etkilenmez.

**Formül:** `X_robust = (X - Q1) / (Q3 - Q1)`

In [ ]:
scaler_robust = RobustScaler()
df_robust_scaled = pd.DataFrame(scaler_robust.fit_transform(df), columns=df.columns)

# Robust ölçeklendirme sonrası dağılımlar
sns.kdeplot(data=df_robust_scaled, palette='viridis')
plt.title('Robust Ölçeklendirme Sonrası Dağılımlar')
plt.show()

print("Robust Ölçeklendirme Sonrası İstatistikler:")
display(df_robust_scaled.describe().round(2))

## 6. Pratik Uygulama: Train-Test Ayrımında Doğru Kullanım

**ÇOK ÖNEMLİ NOT:** Ölçekleyici (scaler), sadece **eğitim (train) verisi** üzerinde `fit` edilmelidir. Test verisinden herhangi bir bilginin (min, max, ortalama gibi) eğitim sürecine sızmasını (`data leakage`) önlemek için, test verisi sadece daha önce eğitilmiş olan bu scaler ile `transform` edilir.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('maas', axis=1)
y = df['maas']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

# 1. Scaler'ı SADECE eğitim verisine göre eğit (fit)
scaler.fit(X_train)

# 2. Hem eğitim hem de test verisini bu scaler ile dönüştür (transform)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Eğitim verisinin şekli:", X_train_scaled.shape)
print("Test verisinin şekli:", X_test_scaled.shape)


## Sonuç: Hangi Yöntemi Ne Zaman Kullanmalı?

| Yöntem | Ne Zaman Kullanılır? | Avantajları | Dezavantajları |
|---|---|---|---|
| **StandardScaler** | **Genel amaçlı, varsayılan tercih.** Özellikle lineer modeller, SVM gibi algoritmalarda. | Aykırı değerlerden daha az etkilenir, dağılımın şeklini korur. | Veriyi belirli bir aralığa sıkıştırmaz. |
| **MinMaxScaler** | Verinin belirli bir aralıkta [0, 1] olması gerektiğinde. **Sinir ağları, görüntü işleme.** | Veriyi sınırlı bir aralığa getirir. | Aykırı değerlere karşı çok hassastır. |
| **RobustScaler** | Veri setinde **önemli sayıda aykırı değer** olduğunda. | Aykırı değerlere karşı dayanıklıdır. | Diğer yöntemler kadar yaygın kullanılmaz. |
